# A1 Prepare catalogs

The purpose of this notebook is to run create a large sample of synthetic cluster members.  Later on these synthetic cluster members will be used to populate synthetic galaxy clusters


## Objectives




## Setup

This notebook relies on two prerequisites:

    * Data files from the previous calculation step. That is the output of the rejection sampling
    * synthetic software package. The host package of this tutorial


## Output

The outuput of this calculation is a very large set of synthetic galaxies sampled from the
    
    * galaxy clusters member model
    * reference line of sight model
    
## Contact

In case of questions, contact me at t.varga@physik.lmu.de

To refactor into above

Synthetic Cluster Generation Tutorial Part 1: Data Preparation

Owner: Tamas Norbert Varga @vargatn

Last Verifed to Run: 2021-09-30 (by @vargatn)

This notebook will describe how to pre-process the DC2 truth catalog to extract catalog cutouts for redMaPPer cluster line-of-sights.

Objectives:

* Using the Generic Catalog Reader we will load the DC2 v1.1.4 truth catalog and the positions and properties of redMaPPer clusters.
* Using healpix disc query, we will then adaptively query the angular vicinity of each cluster
* Add the angular separation from the cluster as a new column
* Save the cutout catalog for future calculation for each cluster. 
* Save three healpix pixel catalogs to give as an estimate of the *field* galaxy population.

Logistics: This notebook is intended to be be run through the This notebook is intended to be run through the JupyterHub NERSC interface available here: https://jupyter-dev.nersc.gov. using the desc_stack_weekly kernel

Other notes:

* Some of the calculations in this notebook are time consuming, please be mindful of the interactive node resources.
* some lines are commented out to avoid overwriting previous file outputs on NERSC CSCRATCH

Output:

The results of this notebook are made available on NERSC at /global/cscratch1/sd/tvarga/DC_DATA/dc2_cluster_sim_cutouts

